In [2]:
import os
import json
import pandas as pd
import traceback

In [3]:
from langchain.chat_models import ChatGooglePalm
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
KEY = os.getenv("GOOGLEPALM_API_KEY")

In [5]:
import google.generativeai as genai

genai.configure(api_key=KEY)
llm = genai.GenerativeModel('gemini-1.5-pro')

AttributeError: module 'google.generativeai' has no attribute 'GenerativeModel'

In [ ]:
llm

'gemini-pro'

In [ ]:
from langchain.llms import GooglePalm
from langchain.chat_models import ChatGooglePalm
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
import PyPDF2

In [ ]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer"
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer"
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer"
    }
}

In [ ]:
TEMPLATE = """
Text:{text}
You are an expert mcq maker. Given above the text, it is your job to \
create a quiz of {number} multiple questions for {subject} students in {tone} tone.
Make sure to format your response like RESPONSE_JSON below and use it as guide.\
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [ ]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number","subject","tone","response_json"],
    template=TEMPLATE
)

In [ ]:
quiz_chain = LLMChain(llm = llm, prompt = quiz_generation_prompt,output_key="quiz",verbose=True)

d:\Projects\MCQ_Generator\venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


ValidationError: 2 validation errors for LLMChain
llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)
llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)

In [ ]:
TEMPLATE2 = """
You are an expert grammerian and writer. Given a multiple choice question for {subject} students.\
you need to evaluate the complexicity of the question and give a complete analysis of the quiz. only use at max 50 words for complexicity
if the quiz is not at par with the cognitive and analytical abilities of the student,\
update the quiz questions which needs to be changed and the change the tone such that it perfectly fit the students ability.\
Quiz_MCQs:
{quiz}

Check from an expert English writer of the above quiz:
"""

In [ ]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject","quiz"],
    template=TEMPLATE2
)

In [ ]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [ ]:
generate_evaluate_chain = SequentialChain(
    chains = [quiz_chain,review_chain],
    input_variables = ["text", "number","subject","tone","response_json"],
    output_variables = ["quiz","review"],
    verbose = True
)

In [ ]:
file_path = 'D:\Projects\MCQ_Generator\data.txt'

In [ ]:
with open(file_path,'r') as file:
    TEXT = file.read()

In [ ]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems contributed 

In [ ]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [ ]:
NUMBER = 5
SUBJECT = "machine learning"
TONE = "simple"

In [ ]:
response = generate_evaluate_chain(
    {
        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
    }
)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used b

d:\Projects\MCQ_Generator\venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Retrying langchain_community.chat_models.google_palm.chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InvalidArgument: 400 * GenerateMessageRequest.model: unexpected model name format
.
Retrying langchain_community.chat_models.google_palm.chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised InvalidArgument: 400 * GenerateMessageRequest.model: unexpected model name format
.
Retrying langchain_community.chat_models.google_palm.chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised InvalidArgument: 400 * GenerateMessageRequest.model: unexpected model name format
.
Retrying langchain_community.chat_models.google_palm.chat_with_retry.<locals>._chat_with_retry in 16.0 seconds as it raised Invali